In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# import surprise
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

#from sklearn import datasets, linear_model
%cd /content/drive/My Drive/Colab Notebooks/Thesis

/content/drive/My Drive/Colab Notebooks/Thesis


In [ ]:
ratings_file = './ml-latest-small/ratings.csv'

In [ ]:
header = ['userId', 'movieId', 'rating','timestamp']
df=pd.read_csv('ml-latest-small/ratings.csv')
df = df.drop(['timestamp'],axis =1)
df.head()
# df.shape

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
n_users = df.userId.unique().shape[0]
n_movies = df.movieId.unique().shape[0]

print ('num of users = ' + str(n_users)+ '| num of movies = ' + str(n_movies))

num of users = 610| num of movies = 9724


In [ ]:
df.movieId.unique().shape

(9724,)

In [ ]:
## Movies ids are not consequtive nor incremental. The next cell fails because although here are 
## 9724 movies, there are ids with larger numbers, for example:
###userId        105.0
###movieId    175401.0
###rating          4.0

#movie ids need to be fixed

In [ ]:
#create ids for movies
moviesids = df[['movieId']].drop_duplicates()
moviesids = moviesids.sort_values(by='movieId')

#s = pd.Series(moviesids.movieId.values.tolist())
moviesids['mid'] = range(0, len(moviesids))
moviesids['mid'] = moviesids['mid']+1
# moviesids

In [ ]:
df = pd.merge(df,moviesids[['movieId','mid']],on='movieId',how='left')
# df

In [ ]:
df = df.drop(columns=['movieId'])
df = df.rename(index=str,columns={'mid':'movieId'})
# df

In [ ]:
# https://towardsdatascience.com/how-to-build-a-simple-recommender-system-in-python-375093c3fb7d
movie_titles = pd.read_csv('ml-latest-small/movies.csv')
# movie_titles.head()

In [ ]:
df = pd.merge(df, movie_titles, on='movieId')
# df.head()

### Dataset analysis/exploration

In [ ]:
ratings = pd.DataFrame(df.groupby('title')['rating'].mean())
# ratings.head()

In [ ]:
ratings['number_of_ratings'] = df.groupby('title')['rating'].count()
# ratings.head()

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# ratings['rating'].hist(bins=50)

In [ ]:
# ratings['number_of_ratings'].hist(bins=60)

In [ ]:
# import seaborn as sns
# sns.jointplot(x='rating', y='number_of_ratings', data=ratings)


In [ ]:
# #location and amount of missing values
# missing_df = pd.DataFrame(df.isnull().sum()).T
# missing_df

In [ ]:
# create utility matrix
movie_matrix = df.pivot_table(index='userId', columns='title', values='rating').fillna(0)
# movie_matrix.head()

In [ ]:
# #location and amount of missing values
# missing_df = pd.DataFrame(movie_matrix.isnull().sum()).T
# missing_df

In [ ]:
# ratings.sort_values('number_of_ratings', ascending=False).head(10)

In [ ]:
# specialist_user_rating = movie_matrix['Specialist, The (1994)']
# specialist_user_rating.head()

In [ ]:
# corr_movie = pd.DataFrame(similar_to_currentmovie, columns=['Correlation'])
# corr_movie.dropna(inplace=True)
# # corr_movie.head()

# corr_movie = corr_movie.join(ratings['number_of_ratings'])
# corr_movie.head()

## cross validation train test split

In [ ]:
train_data, test_data = train_test_split(movie_matrix,test_size=0.25)

In [ ]:
train_data.shape

(457, 5401)

In [ ]:
test_data.shape


(153, 5401)

###  Top K - KNN Predictions

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine',n_neighbors=10, algorithm = 'brute')
model_knn.fit(movie_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [ ]:
# # https://towardsdatascience.com/how-did-we-build-book-recommender-systems-in-an-hour-part-2-k-nearest-neighbors-and-matrix-c04b3c2ef55c
# query_index = np.random.choice(movie_matrix.shape[0])
# distances, indices = model_knn.kneighbors(movie_matrix.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 10)

# for i in range(0, len(distances.flatten())):
#     if i == 0:
#         print('Recommendations for {0}:\n'.format(movie_matrix.index[query_index]))
#     else:
#         print('{0}: {1}, with distance of {2}:'.format(i, movie_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

### User- and Item-based CF

In [ ]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:
#the respective matrix must be produced for items
similarity_matrix_users = fast_similarity(movie_matrix,kind='user')

In [ ]:
similarity_matrix_items = fast_similarity(movie_matrix,kind='item')

In [ ]:
#source: https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/
def predict_topk(ratings, similarity1, kind='user', k=40):
    # start_pred = time.time()
    col_names = ratings.columns.tolist()
    user_names = ratings.index.to_list()
    similarity = similarity1.to_numpy()
    pred = np.zeros(ratings.shape)
    k=k+1
    if kind == 'user':
        for i in range(ratings.shape[0]):
            top_k_users = [np.argsort(similarity[:,i])[:-k-1:-1]]
            for j in range(ratings.shape[1]):
                ################
                pred[i, j] = similarity[i, :][top_k_users].dot(ratings.to_numpy()[:, j][top_k_users]) 
                ## The command above did not work. The solution below however, takes a lot of time to complete
                ## If it takes too long on your laptop, you should fix this
                # jname = col_names[j]
                # dotar = np.zeros(k)
                # c=0
                # for us in top_k_users[0]:
                #     val = ratings.loc[user_names[us],jname]
                #     dotar[c]=val
                #     c+=1
                # ## validate?
                # ################
                # pred[i, j] = similarity[i, :][top_k_users].dot(dotar) 
                pred[i, j] /= np.sum(np.abs(similarity[i, :][top_k_users]))

                # print("user_time_prediction",time.time()-start_pred) 
    if kind == 'item':
        for j in range(ratings.shape[1]):
            top_k_items = [np.argsort(similarity[:,j])[:-k-1:-1]]
            for i in range(ratings.shape[0]):
                ################
                pred[i, j] = similarity[j, :][top_k_items].dot(ratings.to_numpy()[i, :][top_k_items].T) 
                # The command above did not work. The solution below however, takes a lot of time to complete
                # If it takes too long on your laptop, you should fix this
                # dotvar = np.zeros(k)
                # c=0
                # for it in top_k_items[0]:
                #     val = ratings.loc[user_names[i],col_names[it]]
                #     dotvar[c]=val
                #     c+=1
                # ## validate?
                # ################
                # pred[i, j] = similarity[j, :][top_k_items].dot(dotvar.T) 
                
                pred[i, j] /= np.sum(np.abs(similarity[j, :][top_k_items])) 
                # print("item _time_prediction",time.time()-start_pred)       
    
    return pred

In [ ]:
k = 100

In [ ]:
start_pred = time.time()
predicted_values_users = predict_topk(movie_matrix,similarity_matrix_users,'user',k)
end_user_pred = time.time()-start_pred
print("user_time_prediction",end_user_pred)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


user_time_prediction 80.88796830177307


In [ ]:
start=time.time()
predicted_values_items = predict_topk(movie_matrix,similarity_matrix_items,'item',k)
end_item_pred = time.time()-start
print("item _time_prediction",end_item_pred)   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


item _time_prediction 88.318195104599


In [ ]:
tst = movie_matrix.copy()
tst.insert(0, 'order', range(0, 0 + len(tst)))
tst = tst[['order']]
tst = tst.reset_index()

movietst = pd.DataFrame(movie_matrix.columns)
movietst.insert(0, 'order', range(0, 0 + len(movietst)))

results_users= pd.DataFrame()

results_items = pd.DataFrame()

for index,row in test_data.iterrows():
    usr_tst = pd.DataFrame(row)
    #remove the non zero ratings of this user
    usr_tst = usr_tst[usr_tst[index]>0]
    order_of_index= tst.loc[tst['userId']==index,'order'].values.tolist()[0]
    #predicted ratings
    for index1,row1 in usr_tst.iterrows():
        order_of_index1= movietst.loc[movietst['title']==index1,'order'].values.tolist()[0]
        predicted_user = predicted_values_users[order_of_index,order_of_index1]
        real_value = row1[index]
    
        predicted_item = predicted_values_items[order_of_index,order_of_index1]
        
        results_users = results_users.append({'user':index,
                                 'item':index1,
                                 'predicted':predicted_user,
                                 'real_value':real_value},ignore_index=True)
        
        results_items = results_items.append({'user':index,
                                 'item':index1,
                                 'predicted':predicted_item,
                                 'real_value':real_value},ignore_index=True)

In [ ]:
start_res = time.time()
results_users= results_users.drop_duplicates()
end_user_res = time.time()-start_res
print("user _time_prediction",end_user_res) 

results_items= results_items.drop_duplicates()
end_item_res = time.time()-start_res
print("item _time_prediction",end_item_res) 

user _time_prediction 0.016565799713134766
item _time_prediction 0.027730941772460938


##MAE

In [ ]:
print('user-based CF MAE:'+str(mean_absolute_error(results_users.predicted.values.tolist(),results_users.real_value.values.tolist())))

In [ ]:
print('item-based CF MAE:'+str(mean_absolute_error(results_items.predicted.values.tolist(),results_items.real_value.values.tolist())))

In [ ]:
# print('user-based CF MSE:'+str(mean_squared_error(results_users.predicted.values.tolist(),results_users.real_value.values.tolist())))


In [ ]:
# print('item-based CF MSE:'+str(mean_squared_error(results_items.predicted.values.tolist(),results_items.real_value.values.tolist())))

## RMSE

In [ ]:
rmseval = sqrt(mean_squared_error(results_users.real_value.values.tolist(), results_users.predicted.values.tolist()))
print('user-based CF RMSE: '+ str(rmseval))

In [ ]:
rmsevalitem = sqrt(mean_squared_error(results_items.real_value.values.tolist(), results_items.predicted.values.tolist()))
print('item-based CF RMSE: '+ str(rmsevalitem))